In [83]:
from dotenv import load_dotenv
load_dotenv()

import streamlit as st 
import os
import google.generativeai as genai
api_key=os.environ.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

In [84]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-pro")
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from  langchain_core.vectorstores import InMemoryVectorStore
embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [85]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('emergency.pdf')
pdf_doc=loader.load()
pdf_doc

[Document(metadata={'source': 'emergency.pdf', 'page': 0}, page_content=' \n  \n \nEMERGENCY EVACUATION GUIDE \n \nIn the event of a fire or other emergency, seconds count.  The safe, orderly and prompt evacuation of building \noccupants depends on having the physical safety features of a building in operating condition as well as \nhaving an emergency evacuation plan.  The cooperation and participation of every building occupant is \nessential.  Every person that lives and works in a building on campus has an individual responsibility to know \nhow to evacuate in an emergency and to accomplish the evacuation when the fire alarm device sounds or when \ndirected by an authority.  This guide will help you to prepare for emergency situations that might arise in your \nfacility.   \n \nPreplan Your Escape:  \n√ Know the location of fire alarm pull box locations.  \n√ Make sure your floor has at least two unobstructed ways out. \n√ Check the fire exits to ma ke sure they are usable. \n√ Do 

In [86]:
from  langchain.text_splitter import RecursiveCharacterTextSplitter
text_document_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents_pdf=text_document_splitter.split_documents(pdf_doc)
db=InMemoryVectorStore.from_documents(documents=documents_pdf,embedding=embedding)

In [87]:
from langchain.chains import create_history_aware_retriever

In [88]:
retriever_prompt=(
    "Given the following chat history and user question"
    "rephrase the question into a standalone format that captures all relevant context and details from the conversation."
    "create_history_aware_retriever Ensure the reformulated question is clear,"
    "specific, and can be understood independently without requiring reference to the chat history"
)

In [89]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
contextualize_prompt=ChatPromptTemplate.from_messages(
    [
        ('system',retriever_prompt),
        MessagesPlaceholder("history"),
        ("human", "{input}")
    ]
)

In [90]:
retriever=db.as_retriever()
history_aware=create_history_aware_retriever(model, retriever,contextualize_prompt)

In [91]:

prompt = ChatPromptTemplate.from_template(
  """ 
 Answer the following question based only onthe provided context.
 Think step by step before providing the answer.
 <context>{context}</context>
 Question: {input}
"""
)

In [92]:
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(model, prompt)

In [93]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever, question_answer_chain)

response=retrieval_chain.invoke({'input':'what to do ifthe Is the Door Hot'})
response['answer']

'Before opening a door, you should make sure there is no fire on the other side by using the back of your hand to touch the door, \ndoor knob, or door frame.  If any feel hot, don’t open it, there’s probably fire on the other side.  If cool, open the door slowly, \nleave the area and close the door behind you.'

In [94]:
from langchain.chains import create_retrieval_chain
reg_chain=create_retrieval_chain(history_aware,question_answer_chain)
history=[]

In [95]:
from langchain_core.messages.human import HumanMessage
question='If There is a Fire or Fire Alarm what to do'

message1=reg_chain.invoke({'input': question,'history': history})
history.extend([HumanMessage(content=question),message1])
message1

{'input': 'If There is a Fire or Fire Alarm what to do',
 'history': [HumanMessage(content='If There is a Fire or Fire Alarm what to do', additional_kwargs={}, response_metadata={}),
  {...}],
 'context': [Document(id='34696600-ff47-4c1c-a39e-9d0896fe8c20', metadata={'source': 'emergency.pdf', 'page': 0}, page_content='√ Check the fire exits to ma ke sure they are usable. \n√ Do not use the elevators.  They could beco me disabled, trapping you on the fire floor. \n√ Know the location of fire rated stairwells that w ill provide a protected path all the way to the outside. \n√ Learn the sound of your buildings fire alarm.  They could be bells, chimes, horns or a coded gong.  \n√ Post emergency numbers near all telephones.  \n√ If you have fire blankets in your area, know how to use them.  \n \nIf There is a Fire or Fire Alarm: EVERYONE EVACUATES! \n√ If you discover a fire or smok e condition, sound the building alarm by activating the nearest pull \nstation.  Make a follow-up call to Un

In [96]:
message1['answer']

'1. If you discover a fire or smoke condition, sound the building alarm by activating the nearest pull station.\n2. Make a follow-up call to University Police at x333 (632-3333)\n3. Whenever you hear the fire alarm sound, LEAVE  IMMEDIATELY!\n4. Try to help others, if you can do so safely.\n5. leave the area and close the door behind you.'

In [97]:
from langchain_core.messages.human import HumanMessage
question='what to do if  you encounter smoke while escaping'

message1=reg_chain.invoke({'input': question,'history': history})
history.extend([HumanMessage(content=question),message1])
message1

{'input': 'what to do if  you encounter smoke while escaping',
 'history': [HumanMessage(content='If There is a Fire or Fire Alarm what to do', additional_kwargs={}, response_metadata={}),
  {'input': 'If There is a Fire or Fire Alarm what to do',
   'history': [...],
   'context': [Document(id='34696600-ff47-4c1c-a39e-9d0896fe8c20', metadata={'source': 'emergency.pdf', 'page': 0}, page_content='√ Check the fire exits to ma ke sure they are usable. \n√ Do not use the elevators.  They could beco me disabled, trapping you on the fire floor. \n√ Know the location of fire rated stairwells that w ill provide a protected path all the way to the outside. \n√ Learn the sound of your buildings fire alarm.  They could be bells, chimes, horns or a coded gong.  \n√ Post emergency numbers near all telephones.  \n√ If you have fire blankets in your area, know how to use them.  \n \nIf There is a Fire or Fire Alarm: EVERYONE EVACUATES! \n√ If you discover a fire or smok e condition, sound the buildin

In [98]:
message1['answer']

'1. If you encounter smoke while escaping, crawl or get as low as you can.\n2. The cleanest air will be within 1 to 2 feet of the floor.\n3. If the main exit is blocked by fire or smoke, you should use your alternate route.\n4. If this is not feasible, go back in your room to wait for rescue.'

# pass the history not mannually